<a href="https://colab.research.google.com/github/Codewiz22/AES/blob/main/HMAC_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import hmac
import hashlib

class HMACManager:
    """
    A professional-grade HMAC implementation for academic study.
    Standard: HMAC using SHA-256 (RFC 2104).

    HMAC provides a way to verify both the data integrity and the
    authenticity of a message using a shared secret key.
    """
    def __init__(self, key=None):
        # HMAC keys should ideally be the same size as the hash output.
        # For SHA-256, a 32-byte (256-bit) key is standard.
        self.key = key if key else os.urandom(32)

    def generate_tag(self, message: str) -> bytes:
        """
        Generates an HMAC tag for a given message.
        """
        message_bytes = message.encode('utf-8')

        # hmac.new(key, msg, digestmod) implements the nested hashing:
        # HMAC(K, m) = H((K ^ opad) || H((K ^ ipad) || m))
        h = hmac.new(self.key, message_bytes, hashlib.sha256)
        return h.digest()

    def verify_tag(self, message: str, received_tag: bytes) -> bool:
        """
        Verifies a received HMAC tag against a message.
        Uses constant-time comparison to prevent timing attacks.
        """
        expected_tag = self.generate_tag(message)

        # hmac.compare_digest is a 'constant-time' comparison.
        # This prevents attackers from guessing the tag byte-by-byte based on response time.
        return hmac.compare_digest(expected_tag, received_tag)

# --- Usage Example for Week 11 / Supplemental Module ---
if __name__ == "__main__":
    print("--- HMAC-SHA256 Authentication Demo ---")

    # 1. Setup
    shared_secret = b"super-secret-300-level-cryptography-key"
    auth_manager = HMACManager(shared_secret)

    # 2. Sender side
    original_message = "Payload: Transfer $5000 to Account 12345"
    tag = auth_manager.generate_tag(original_message)

    print(f"Message:   {original_message}")
    print(f"HMAC Tag:  {tag.hex()}")

    # 3. Receiver side (Successful Verification)
    is_valid = auth_manager.verify_tag(original_message, tag)
    print(f"\nVerification (Original): {'✅ SUCCESS' if is_valid else '❌ FAILED'}")

    # 4. Receiver side (Tamper Attempt)
    tampered_message = "Payload: Transfer $9000 to Account 12345"
    is_valid_tamper = auth_manager.verify_tag(tampered_message, tag)
    print(f"Verification (Tampered): {'✅ SUCCESS' if is_valid_tamper else '❌ FAILED'}")

    # 5. Receiver side (Wrong Key Attempt)
    wrong_manager = HMACManager(os.urandom(32))
    is_valid_key = wrong_manager.verify_tag(original_message, tag)
    print(f"Verification (Wrong Key): {'✅ SUCCESS' if is_valid_key else '❌ FAILED'}")

--- HMAC-SHA256 Authentication Demo ---
Message:   Payload: Transfer $5000 to Account 12345
HMAC Tag:  001a8fee767c905b565e41f1c1d2525d18cc0721cc584c6865c3853b99584ad2

Verification (Original): ✅ SUCCESS
Verification (Tampered): ❌ FAILED
Verification (Wrong Key): ❌ FAILED
